# Regression

In [31]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
# Basic ANN
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self,epoch, logs=None):
        if logs['accuracy'] >0.90:
            print("Accuracy greater than 90%. Stopping Training.")
            self.model.stop_training=True
            
def val_dnn_model(epochs, X_train, Y_train, X_val, Y_val, callbacks=None):
    model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(1, activation='sigmoid')])

    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=epochs, callbacks=callbacks)

    return history, model


In [32]:

train = tfds.load('german_credit_numeric', split=['train'], batch_size=-1, as_supervised=True)
X=tfds.as_numpy(train[0][0])
Y=tfds.as_numpy(train[0][1])
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3)
hist_regularized, model_regularized = val_dnn_model(100, X_train, Y_train, X_test, Y_test, callbacks=[EarlyStoppingCallback()])
print(f"Training Set:   {model_regularized.evaluate(X_train, Y_train)}")
print(f"Validation Set: {model_regularized.evaluate(X_test, Y_test)}")

Epoch 1/100
22/22 [==============================] - 1s 15ms/step - loss: 1.4392 - accuracy: 0.6114 - val_loss: 0.7750 - val_accuracy: 0.6233
Epoch 2/100
22/22 [==============================] - 0s 9ms/step - loss: 1.1206 - accuracy: 0.6157 - val_loss: 0.6447 - val_accuracy: 0.7033
Epoch 3/100
22/22 [==============================] - 0s 8ms/step - loss: 0.8553 - accuracy: 0.6257 - val_loss: 0.6501 - val_accuracy: 0.6900
Epoch 4/100
22/22 [==============================] - 0s 8ms/step - loss: 0.7674 - accuracy: 0.6686 - val_loss: 0.7009 - val_accuracy: 0.7067
Epoch 5/100
22/22 [==============================] - 0s 9ms/step - loss: 0.7329 - accuracy: 0.6914 - val_loss: 0.6520 - val_accuracy: 0.7067
Epoch 6/100
22/22 [==============================] - 0s 9ms/step - loss: 0.6660 - accuracy: 0.6971 - val_loss: 0.5500 - val_accuracy: 0.7133
Epoch 7/100
22/22 [==============================] - 0s 10ms/step - loss: 0.6634 - accuracy: 0.7043 - val_loss: 0.5326 - val_accuracy: 0.7267
Epoch 8/100

In [40]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.model_selection import train_test_split
def vtoc_regression_model(norm, model_type):
    if model_type=='linear':
        model = Sequential()
        #model.add(Flatten())
        model.add(Dense(1))
    else:
        model = Sequential()
        model.add(Dense(64, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(1))

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.1), loss='mean_absolute_error')

    return model


def eval_regression_data(dataset, target):
    dataset = dataset.astype('float32')
    dataset.isna().sum()
    dataset=dataset.dropna()
    #dataset['Origin'] = dataset['Origin'].map({1.0: 'USA', 2.0: 'Europe', 3.0: 'Japan'})
    Y=dataset[target]
    X=dataset.loc[:, dataset.columns != target]
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3)
    horsepower = np.array(X_train['Horsepower']).astype('float32')

    horsepower_normalizer = Normalization(input_shape=[1,], axis=None)
    horsepower_normalizer.adapt(horsepower)
    
    linear_model=vtoc_regression_model(horsepower_normalizer, model_type='linear')
    history = linear_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=100, verbose=2)
    test_results = linear_model.predict(X_test)
    return test_results

In [41]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)
eval_regression_data(raw_dataset, 'MPG')


Epoch 1/100
9/9 - 0s - loss: 231.0550 - val_loss: 290.7013 - 349ms/epoch - 39ms/step
Epoch 2/100
9/9 - 0s - loss: 164.8466 - val_loss: 26.5685 - 119ms/epoch - 13ms/step
Epoch 3/100
9/9 - 0s - loss: 89.5644 - val_loss: 49.4564 - 100ms/epoch - 11ms/step
Epoch 4/100
9/9 - 0s - loss: 32.7282 - val_loss: 28.5107 - 103ms/epoch - 11ms/step
Epoch 5/100
9/9 - 0s - loss: 22.5977 - val_loss: 8.0420 - 79ms/epoch - 9ms/step
Epoch 6/100
9/9 - 0s - loss: 8.5712 - val_loss: 13.5126 - 81ms/epoch - 9ms/step
Epoch 7/100
9/9 - 0s - loss: 16.3905 - val_loss: 9.2972 - 67ms/epoch - 7ms/step
Epoch 8/100
9/9 - 0s - loss: 35.3781 - val_loss: 11.4659 - 80ms/epoch - 9ms/step
Epoch 9/100
9/9 - 0s - loss: 14.0517 - val_loss: 18.2754 - 77ms/epoch - 9ms/step
Epoch 10/100
9/9 - 0s - loss: 16.9186 - val_loss: 32.8455 - 84ms/epoch - 9ms/step
Epoch 11/100
9/9 - 0s - loss: 11.7090 - val_loss: 9.9018 - 142ms/epoch - 16ms/step
Epoch 12/100
9/9 - 0s - loss: 10.5167 - val_loss: 43.4549 - 136ms/epoch - 15ms/step
Epoch 13/100
9

array([[  0.14522398],
       [ -9.3601055 ],
       [-29.113241  ],
       [ -8.973978  ],
       [-41.372612  ],
       [-15.517832  ],
       [-34.509792  ],
       [  2.6275582 ],
       [ -4.1425376 ],
       [ -8.209433  ],
       [-46.33242   ],
       [-13.186731  ],
       [-46.559914  ],
       [-14.423948  ],
       [-11.954599  ],
       [ -6.548349  ],
       [-20.98937   ],
       [-43.63129   ],
       [-43.856377  ],
       [ -3.3977714 ],
       [-40.002396  ],
       [-38.01322   ],
       [-13.034712  ],
       [  5.133372  ],
       [-57.685776  ],
       [ -1.5658737 ],
       [-34.57497   ],
       [-12.189146  ],
       [-29.044123  ],
       [ -8.435602  ],
       [ -5.9060683 ],
       [-39.161034  ],
       [  6.5103173 ],
       [  3.2081475 ],
       [-32.611668  ],
       [  3.7958436 ],
       [ 10.678658  ],
       [ -1.5126663 ],
       [-24.098246  ],
       [-28.153078  ],
       [-10.764357  ],
       [-17.608385  ],
       [-39.674133  ],
       [-23

# Build Network from scratch

In [43]:
# Feedforward Neural Network
import tensorflow as tf
def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def create_dataset(xs, ys, n_classes=10):
    xs = tf.cast(xs, tf.float32) / 255.0
    ys = tf.cast(ys, tf.float32)
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)).map(preprocess).shuffle(len(ys)).batch(128)

def val_nn(training_inputs_data, training_outputs_data, test_inputs):
    tf.compat.v1.disable_eager_execution()
    training_inputs = tensorflow.compat.v1.placeholder(shape=[None, 3], dtype=tensorflow.float32)  
    training_outputs = tensorflow.compat.v1.placeholder(shape=[None, 1], dtype=tensorflow.float32) #Desired outputs for each input  
    weights = tensorflow.Variable(initial_value=[[.3], [.1], [.8]], dtype=tensorflow.float32)  
    bias = tensorflow.Variable(initial_value=[[1]], dtype=tensorflow.float32)  

    af_input = tensorflow.matmul(training_inputs, weights) + bias  
  
    # Activation function of the output layer neuron  
    predictions = tensorflow.nn.sigmoid(af_input)  
    # Measuring the prediction error of the network after being trained  
    prediction_error = tensorflow.reduce_sum(training_outputs - predictions)  
    # Minimizing the prediction error using gradient descent optimizer  
    
    train_op = tensorflow.compat.v1.train.GradientDescentOptimizer(learning_rate=0.05).minimize(prediction_error) 
    # Creating a TensorFlow Session  
    sess = tensorflow.compat.v1.Session()  
    # Initializing the TensorFlow Variables (weights and bias)  
    sess.run(tensorflow.compat.v1.global_variables_initializer())  
    
    # Training loop of the neural network  
    for step in range(10000):  
        sess.run(fetches=train_op, feed_dict={training_inputs: training_inputs_data, training_outputs: training_outputs_data})  
        # Class scores of some testing data  
    score= sess.run(fetches=predictions, feed_dict={training_inputs: [[248, 80, 68], [0, 0, 255]]})
    tf.compat.v1.disable_eager_execution()
    predictions =  sess.run(fetches=test_inputs)
    # Closing the TensorFlow Session to free resources  
    sess.close()
    return predictions

In [44]:
import tensorflow
X_train = [[255, 0, 0], [248, 80, 68], [0, 0, 255],[67, 15, 210]]  
Y_train = [[1], [1], [0], [0]] 
X_test = [[5, 2, 13], [7, 9, 0]]
tf.compat.v1.disable_eager_execution()
X_test=tensorflow.convert_to_tensor(value=X_test, dtype=tensorflow.float32)
val_nn(X_train, Y_train, X_test)

array([[ 5.,  2., 13.],
       [ 7.,  9.,  0.]], dtype=float32)

In [45]:
# Backprop Neural Network
from random import seed
from random import random
from math import exp
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network


# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = 1.0 / (1.0 + exp(-activation))
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(neuron['output'] - expected[j])
        for j in range(len(layer)):
            neuron = layer[j]
            transfer_derivative = neuron['output'] * (1.0 - neuron['output'])
            neuron['delta'] = errors[j] * transfer_derivative


# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] -= l_rate * neuron['delta']


# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))


def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [46]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836, 2.550537003, 0],
           [1.465489372, 2.362125076, 0],
           [3.396561688, 4.400293529, 0],
           [1.38807019, 1.850220317, 0],
           [3.06407232, 3.005305973, 0],
           [7.627531214, 2.759262235, 1],
           [5.332441248, 2.088626775, 1],
           [6.922596716, 1.77106367, 1],
           [8.675418651, -0.242068655, 1],
           [7.673756466, 3.508563011, 1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
           [{'weights': [0.2550690257394217, 0.49543508709194095]},
            {'weights': [0.4494910647887381, 0.651592972722763]}]]

for row in dataset:
    prediction = predict(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

row = [1, 0, None]
output = forward_propagate(network, row)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': 0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': -0.0026279652850863837}]
[{'weights

# Build CNN network using Pure Python

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from scipy.signal import convolve2d

# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32')# / 255.0
X_test = X_test.astype('float32')# / 255.0

# Flatten labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train.flatten())
y_test = label_encoder.transform(y_test.flatten())

class SimpleCNNCifar:
    def __init__(self):
        self.weights = {
            'conv1': np.random.randn(3, 3, 3, 32) / (3 * 3 * 3),
            'batch_norm1_gamma': np.ones((1, 1, 1, 32)),
            'batch_norm1_beta': np.zeros((1, 1, 1, 32)),
            'conv2': np.random.randn(3, 3, 32, 32) / (3 * 3 * 32),
            'batch_norm2_gamma': np.ones((1, 1, 1, 32)),
            'batch_norm2_beta': np.zeros((1, 1, 1, 32)),
            'conv3': np.random.randn(3, 3, 32, 64) / (3 * 3 * 32),
            'batch_norm3_gamma': np.ones((1, 1, 1, 64)),
            'batch_norm3_beta': np.zeros((1, 1, 1, 64)),
            'conv4': np.random.randn(3, 3, 64, 64) / (3 * 3 * 64),
            'batch_norm4_gamma': np.ones((1, 1, 1, 64)),
            'batch_norm4_beta': np.zeros((1, 1, 1, 64)),
            'conv5': np.random.randn(3, 3, 64, 128) / (3 * 3 * 64),
            'batch_norm5_gamma': np.ones((1, 1, 1, 128)),
            'batch_norm5_beta': np.zeros((1, 1, 1, 128)),
            'conv6': np.random.randn(3, 3, 128, 128) / (3 * 3 * 128),
            'batch_norm6_gamma': np.ones((1, 1, 1, 128)),
            'batch_norm6_beta': np.zeros((1, 1, 1, 128)),
            'dense1': np.random.randn(2048, 10) / 2048,
            'dense1_bias': np.zeros((1, 10)),
            'fc': np.random.randn(2048, 10) / 2048,
            'fc_bias': np.zeros((1, 10)),
        }
        self.epsilon = 1e-5

    def forward_pass(self, images):
        conv1 = self.convolution(images, self.weights['conv1'])
        print(conv1.shape)
        activation1 = self.relu_activation(conv1)
        print(activation1.shape)
        batch_norm1 = self.batch_normalization(activation1, self.weights['batch_norm1_gamma'], self.weights['batch_norm1_beta'])
        print(batch_norm1.shape)
        conv2 = self.convolution(activation1, self.weights['conv2'])
        
        activation2 = self.relu_activation(conv2)
        batch_norm2 = self.batch_normalization(activation2, self.weights['batch_norm2_gamma'], self.weights['batch_norm2_beta'])
        pool1 = self.max_pooling(batch_norm2)
        print(pool1.shape)
        conv3 = self.convolution(pool1, self.weights['conv3'])
        batch_norm3 = self.batch_normalization(conv3, self.weights['batch_norm3_gamma'], self.weights['batch_norm3_beta'])
        activation3 = self.relu_activation(batch_norm3)
        print(activation3.shape)
        conv4 = self.convolution(activation3, self.weights['conv4'])
        batch_norm4 = self.batch_normalization(conv4, self.weights['batch_norm4_gamma'], self.weights['batch_norm4_beta'])
        activation4 = self.relu_activation(batch_norm4)
        pool2 = self.max_pooling(activation4)
        print(pool2.shape)
        conv5 = self.convolution(pool2, self.weights['conv5'])
        batch_norm5 = self.batch_normalization(conv5, self.weights['batch_norm5_gamma'], self.weights['batch_norm5_beta'])
        activation5 = self.relu_activation(batch_norm5)

        conv6 = self.convolution(activation5, self.weights['conv6'])
        batch_norm6 = self.batch_normalization(conv6, self.weights['batch_norm6_gamma'], self.weights['batch_norm6_beta'])
        activation6 = self.relu_activation(batch_norm6)
        pool3 = self.max_pooling(activation6)
        print(pool3.shape)
        flatten = pool3.reshape((pool3.shape[0], -1))
        dense1 = np.dot(flatten, self.weights['dense1']) + self.weights['dense1_bias']
        print(dense1.shape)
        return dense1

    def batch_normalization(self, x, gamma, beta):
        mean = np.mean(x, axis=(0, 1, 2), keepdims=True)
        variance = np.var(x, axis=(0, 1, 2), keepdims=True)
        x_normalized = (x - mean) / np.sqrt(variance + self.epsilon)
        return gamma * x_normalized + beta


    def max_pooling(self, image, pool_size=(2, 2)):
        return np.max(image.reshape((image.shape[0], image.shape[1] // pool_size[0], pool_size[0], image.shape[2] // pool_size[1], pool_size[1], image.shape[3])), axis=(2, 4))

    def relu_activation(self, x):
        return np.maximum(0, x)


    def convolution(self, image, kernel):
        result = [
            np.sum(
                [convolve2d(image[i, :, :, c], kernel[:, :, c, j], mode='same', boundary='symm')[:, :, np.newaxis]
                 for c in range(image.shape[-1])]
            , axis=0)
            for i in range(len(image))
            for j in range(kernel.shape[-1])
        ]
        result = np.array(result)
        result = result.reshape((len(image), image.shape[1], image.shape[2], -1))
        return result

    def softmax_activation(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def fit(self, X, y, epochs=5, learning_rate=0.01, batch_size=64):
        for epoch in range(epochs):
            for i in range(0, len(X), batch_size):
                end = min(i + batch_size, len(X))
                batch_images = X[i:end]
                batch_labels = y[i:end].astype(int)  # Convert labels to integers

                # Forward pass
                predictions = self.forward_pass(batch_images)

                # Compute loss
                loss = self.compute_loss(predictions, batch_labels)

                # Backward pass
                gradients = self.backward_pass(batch_images, predictions, batch_labels)

                # Update weights
                self.update_weights(gradients, learning_rate)

    def compute_loss(self, predictions, labels):
        return np.mean(-np.log(predictions[np.arange(len(predictions)), labels]))



    def backward_pass(self, images, predictions, labels):
        gradients = {}
        num_samples = len(images)
    
        output_gradients = predictions.copy()
        output_gradients[np.arange(num_samples), labels] -= 1
        output_gradients /= num_samples
    
        gradients['fc'] = np.dot(images.reshape((len(images), -1)).T, output_gradients)
        gradients['fc_bias'] = np.sum(output_gradients, axis=0, keepdims=True)
    
        fc_gradients = np.dot(output_gradients, self.weights['fc'].T)
        fc_gradients_reshaped = fc_gradients.reshape((len(images), 4, 4, 128))
    
        pool3_gradients = fc_gradients_reshaped.repeat(2, axis=1).repeat(2, axis=2)
        
        conv6_gradients = np.zeros_like(images)
        for i in range(3):
            for j in range(128):
                conv6_gradients[:, :, :, i:i+3, :, :] += np.sum(images[:, np.newaxis, :, i:i+3, :, :] * pool3_gradients[:, j, :, np.newaxis, np.newaxis, :], axis=(2, 3, 4))

        conv6_gradients = np.sum(conv6_gradients, axis=(1, 2, 3))
        conv6_gradients = conv6_gradients.reshape((3, 128, len(pool3_gradients), pool3_gradients.shape[2], pool3_gradients.shape[3]))
        conv6_gradients = np.sum(conv6_gradients, axis=0)
        conv6_gradients = conv6_gradients[:, :, np.newaxis, :, :]

    
        gradients['conv6'] = np.zeros_like(self.weights['conv6'])
        for i in range(128):
            for j in range(128):
                gradients['conv6'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv6_gradients[:, j, :, :], axis=(0, 2, 3)
                )
    
        pool3_gradients = conv6_gradients.repeat(2, axis=1).repeat(2, axis=2)
        conv5_gradients = pool3_gradients * (self.weights['conv5'] > 0)
    
        gradients['conv5'] = np.zeros_like(self.weights['conv5'])
        for i in range(64):
            for j in range(128):
                gradients['conv5'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv5_gradients[:, j, :, :], axis=(0, 2, 3)
                )

        pool2_gradients = conv5_gradients.repeat(2, axis=1).repeat(2, axis=2)
        conv4_gradients = pool2_gradients * (self.weights['conv4'] > 0)
    
        gradients['conv4'] = np.zeros_like(self.weights['conv4'])
        for i in range(64):
            for j in range(64):
                gradients['conv4'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv4_gradients[:, j, :, :], axis=(0, 2, 3)
                )
    
        pool2_gradients = conv4_gradients.repeat(2, axis=1).repeat(2, axis=2)
        conv3_gradients = pool2_gradients * (self.weights['conv3'] > 0)
    
        gradients['conv3'] = np.zeros_like(self.weights['conv3'])
        for i in range(32):
            for j in range(64):
                gradients['conv3'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv3_gradients[:, j, :, :], axis=(0, 2, 3)
                )
    
        pool1_gradients = conv3_gradients.repeat(2, axis=1).repeat(2, axis=2)
        conv2_gradients = pool1_gradients * (self.weights['conv2'] > 0)
    
        gradients['conv2'] = np.zeros_like(self.weights['conv2'])
        for i in range(32):
            for j in range(32):
                gradients['conv2'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv2_gradients[:, j, :, :], axis=(0, 2, 3)
                )

        pool1_gradients = conv2_gradients.repeat(2, axis=1).repeat(2, axis=2)
        conv1_gradients = pool1_gradients * (self.weights['conv1'] > 0)
    
        gradients['conv1'] = np.zeros_like(self.weights['conv1'])
        for i in range(3):
            for j in range(32):
                gradients['conv1'][:, :, :, j] += np.sum(
                    images[:, i:i+3, :, :] * conv1_gradients[:, j, :, :], axis=(0, 2, 3)
                )
    
        return gradients


# Convert tensors to Pandas DataFrame using threading
def convert_to_dataframe(X, y, start, end, result, cnn_model):
    data = []

    for i in range(start, end):
        image = X[i].reshape(-1, 32, 32, 3)
        label = y[i]

        # Forward pass through the CNN to get features
        features = cnn_model.forward_pass(np.expand_dims(image, axis=0))

        data.append(np.concatenate([features.flatten(), [int(label)]]))

    result.extend(data)

# Function to use threading effectively
def tensors_to_dataframe(X, y, cnn_model, num_threads=4, batch_size=64):
    data = []
    threads = []

    for i in range(0, len(X), batch_size * num_threads):
        for j in range(num_threads):
            start = i + j * batch_size
            end = min(i + (j + 1) * batch_size, len(X))
            thread_result = []
            thread = threading.Thread(target=convert_to_dataframe, args=(X[start:end], y[start:end], start, end, thread_result, cnn_model))
            thread.daemon = True  # Set daemon attribute to True
            thread.start()
            threads.append((thread, thread_result))

    for thread, thread_result in threads:
        try:
            thread.join()
        except Exception as e:
            pass  # Do nothing on exception

        data.extend(thread_result)

    # Create Pandas DataFrame
    columns = [f'feature_{i}' for i in range(data[0].shape[0] - 1)] + ['label']
    df = pd.DataFrame(data, columns=columns)

    return df

# Train CNN model
cnn_model = SimpleCNNCifar()
cnn_model.fit(X_train, y_train, epochs=5)

# Convert tensors to Pandas DataFrame using threading
df_train_cnn = tensors_to_dataframe(X_train, y_train, cnn_model, batch_size=64)

# Split the data into train and test sets
df_train_cnn, df_test_cnn = train_test_split(df_train_cnn, test_size=0.2, random_state=42)

# Separate features and labels
X_train_cnn, y_train_cnn = df_train_cnn.iloc[:, :-1], df_train_cnn['label']
X_test_cnn, y_test_cnn = df_test_cnn.iloc[:, :-1], df_test_cnn['label']

# Continue with your desired classification model or further processing using the CNN features.
# For example, if you want to use the SimpleCNNCifar model for classification:

# Initialize and train SimpleCNNCifar model
cnn_model_cifar = SimpleCNNCifar()
cnn_model_cifar.fit(X_train_cnn, y_train_cnn, epochs=5, learning_rate=0.01)

# Ensure the number of samples in X_test_cnn matches y_test_cnn
X_test_cnn = df_test_cnn.iloc[:, :-1].values.reshape(-1, 32, 32, 3)
y_test_cnn = df_test_cnn['label']

# Make predictions on the test set
y_pred_cnn_cifar = np.argmax(cnn_model_cifar.forward_pass(X_test_cnn), axis=1)

# Calculate accuracy
accuracy_cnn_cifar = accuracy_score(y_test_cnn, y_pred_cnn_cifar)
print(f'SimpleCNNCifar Accuracy: {accuracy_cnn_cifar:.4f}')


(64, 32, 32, 32)
(64, 32, 32, 32)
(64, 32, 32, 32)
(64, 16, 16, 32)
(64, 16, 16, 64)
(64, 8, 8, 64)


In [4]:
import tensorflow as tf
import numpy as np 

# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Build the CNN model using subclassed layers
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        # Convolutional layer 1
        self.conv1 = tf.Variable(tf.random.normal([3, 3, 3, 32]))
        # Convolutional layer 2
        self.conv2 = tf.Variable(tf.random.normal([3, 3, 32, 64]))
        # Fully connected layer
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.Variable(tf.random.normal([8 * 8 * 64, 256]))
        # Output layer
        self.output_layer = tf.Variable(tf.random.normal([256, 10]))

    def call(self, x, training=True):
        # Convolutional layer 1
        conv1 = tf.nn.relu(tf.nn.conv2d(x, self.conv1, strides=[1, 1, 1, 1], padding='SAME'))
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

        # Convolutional layer 2
        conv2 = tf.nn.relu(tf.nn.conv2d(pool1, self.conv2, strides=[1, 1, 1, 1], padding='SAME'))
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

        # Flatten
        flattened = self.flatten(pool2)

        # Fully connected layer
        fc = tf.nn.relu(tf.matmul(flattened, self.fc))

        # Output layer
        output = tf.matmul(fc, self.output_layer)

        return output

# Instantiate the model
model = MyModel()

# Define training parameters
learning_rate = 0.001
epochs = 5
batch_size = 64

# Use tf.data.Dataset for input pipeline parallelization
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)

# Define loss and optimizer
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

# Training the model
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        with tf.device('/device:GPU:0'):  # Specify the GPU device
            with tf.GradientTape() as tape:
                logits = model(x_batch, training=True)
                current_loss = loss_fn(y_batch, logits)

            gradients = tape.gradient(current_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Calculate training loss and accuracy
    train_loss = loss_fn(y_train, model(x_train, training=False))
    train_accuracy = np.mean(np.argmax(model(x_train, training=False), axis=1) == np.argmax(y_train, axis=1))

    print(f'Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}')

# Calculate test accuracy
test_accuracy = np.mean(np.argmax(model(x_test, training=False), axis=1) == np.argmax(y_test, axis=1))
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/5, Loss: 1422.8845, Accuracy: 0.3310
Epoch 2/5, Loss: 818.0657, Accuracy: 0.3730
Epoch 3/5, Loss: 544.9580, Accuracy: 0.3941
Epoch 4/5, Loss: 376.3708, Accuracy: 0.4138
Epoch 5/5, Loss: 278.3134, Accuracy: 0.4148
Test Accuracy: 0.3905


# Tensor Basics

In [2]:
import tensorflow as tf
X_train = [[255, 0, 0], [248, 80, 68], [0, 0, 255],[67, 15, 210]]  
Y_train = [[1], [1], [0], [0]] 
X_test = [[5, 2, 13], [7, 9, 0]]
xt=tf.constant(X_train)
xt.shape, tf.rank(xt)
y1 = tf.random.uniform(shape=[5, 300], minval=0, maxval=1, dtype=tf.float64)
y2 = tf.random.uniform(shape=[5, 300], minval=0, maxval=1, dtype=tf.float64)
prob4=tf.matmul(y1, tf.transpose(y2))
prob5=tf.tensordot(y1, tf.transpose(y2), axes=1)
y6 = tf.random.uniform(shape=[224, 224, 3], minval=0, maxval=1, dtype=tf.float64)
prob6=tf.math.reduce_max(y6, axis=0)
y7 = tf.random.uniform(shape=[1, 224, 224, 3], minval=0, maxval=1, dtype=tf.float64)
prob7 = tf.squeeze(y7, axis=0)
y8 = tf.random.uniform(shape=[10], minval=0, maxval=10, dtype=tf.int64)
prob9=tf.math.argmax(y8)
prob10=tf.one_hot(y8, depth=10)

In [3]:
tf.constant(X_train)
a = tf.linspace(-1, 1, 10)
a_new = tf.expand_dims(a, axis=1)
a_new
a_transpose=tf.transpose(a_new)
a_randint = tf.random.uniform(shape=[3,4], minval=1, maxval=10, dtype=tf.int32)
a_randn = tf.random.normal(shape=[3,4])
a_zeros = tf.zeros(shape=[3,4])
a_ones = tf.ones(shape=[3,4])
a_fives = tf.fill([3,4], 5.2)
a_eye = tf.eye(5)
x = tf.expand_dims(tf.Variable([1, 2, 0, 4]), axis=1)
mask = x >= 2
a_slice = tf.boolean_mask(x, mask)
masked = tf.greater(x,1)
zeros=tf.zeros_like(x)
a_masked = tf.where(masked, x, zeros)
a_sq_eye = tf.reshape(a_eye, [25])
a_new_transpose = tf.squeeze(a_transpose)
a_concat = tf.concat((tf.cast(a_new, tf.float32), tf.cast(tf.expand_dims(a_sq_eye, axis=1), tf.float32)), axis=0)
a_matmul = tf.matmul(tf.transpose(a_masked), x)

numpy_arr = np.array([10.0, 11.0, 12.0, 13.0])
from_numpy_to_tensor = tf.convert_to_tensor(numpy_arr)
a_min = tf.reduce_min(x)
a_max = tf.reduce_max(x, axis=1)
a_randint, a_randn, a_zeros, a_fives, a_ones, a_eye, a_randint.shape, a_randn.dtype, a_slice, a_masked, a_sq_eye, a_transpose, a_new_transpose, a_concat, a_matmul, from_numpy_to_tensor, a_min, a_max

(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
 array([[1, 9, 3, 2],
        [1, 3, 9, 7],
        [8, 6, 9, 4]])>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[-1.5040163 , -0.8377314 ,  0.5743124 , -0.833165  ],
        [-0.693856  , -1.2710524 , -0.69161797, -0.3762566 ],
        [-0.0787261 ,  0.33580709,  0.59934694, -0.15102491]],
       dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[5.2, 5.2, 5.2, 5.2],
        [5.2, 5.2, 5.2, 5.2],
        [5.2, 5.2, 5.2, 5.2]], dtype=float32)>,
 <tf.Tensor: shape=(3, 4), dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
 array([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        